In [1]:
# Chris Hicks 2020
# https://www.kaggle.com/c/digit-recognizer
#
# The data files train.csv and test.csv contain gray-scale images of hand-drawn digits, from zero through nine.
# Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. 
# Each pixel has a single pixel-value associated with it, an integer between 0 and 255, inclusive.
# The training data set, (train.csv), has 785 columns. The first column, called "label", is the digit that was 
# drawn by the user. The rest of the columns contain the pixel-values of the associated image.
#
# test.csv is the same as the training set, except that it does not contain the "label" column.
import math
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
#from tensorflow.python.framework import ops
#from tf_utils import load_dataset, random_mini_batches, convert_to_one_hot, predict

In [2]:
# Load the train dataset, returning numpy arrays comprising the respective train data, 
# train labels and test data.
# Inputs: filename of training data in CSV format
#         tuple (train, cv, test) enciding data split as whole digit percentages e.g. 80-10-10 
# Output: numpy arrays comprising the respective train data, train labels, test data and test labels
def load_dataset(train_filename, split=(90,10)):
    # Load training data
    train_data = pd.read_csv(train_filename)
    labels = train_data['label']
    labels = labels.to_numpy()
    train_data = train_data.drop('label', axis=1)
    train_data = train_data.to_numpy()
    
    # Split boundaries
    n_train = math.floor(train_data.shape[0]*(split[0]/100))
    
    train_X = train_data[:n_train]
    train_labels = labels[:n_train]
    
    cv_X = train_data[n_train:]
    cv_labels = labels[n_train:]
    
    return train_X, train_labels, cv_X, cv_labels

In [3]:
#  Creates a list of random minibatches from (X, Y), each of size mini_batch_size
def random_mini_batches(X, Y, mini_batch_size = 64):
    
    m = X.shape[1]                  # number of training examples
    mini_batches = []
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((Y.shape[0], m))

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [4]:
# Convert the column vector Y comprising n_classes possible values into a one-hot encoded array.
# Returns: Yxn_classes dimensional one-hot encoded array
def convert_to_one_hot(Y, n_classes):
    return np.eye(n_classes)[Y.reshape(-1)].T

In [5]:
# Create TF placeholders
# Returns:
#    X : placeholder for the data input, of shape [n_x, None] and dtype "tf.float32"
#    Y : placeholder for the input labels, of shape [n_y, None] and dtype "tf.float32"
def create_placeholders(n_x, n_y):
    X = tf.placeholder(tf.float32, (n_x, None), name = "X")
    Y = tf.placeholder(tf.float32, (n_y, None), name = "Y")
    
    return X, Y

In [6]:
# Initialise parameters for neural network graph
# Inputs:
#     image_size : total number of flattened pixel values in image
#     n_classes : number of output classes in network
# Returns:
#    parameters W1,...WL, b1,...,bL
def initialise_parameters(image_size, n_classes):
    
    W1 = tf.get_variable("W1", [25,image_size], initializer = tf.contrib.layers.xavier_initializer())
    b1 = tf.get_variable("b1", [25,1], initializer = tf.zeros_initializer())
    W2 = tf.get_variable("W2", [12,25], initializer = tf.contrib.layers.xavier_initializer())
    b2 = tf.get_variable("b2", [12,1], initializer = tf.zeros_initializer())
    W3 = tf.get_variable("W3", [n_classes,12], initializer = tf.contrib.layers.xavier_initializer())
    b3 = tf.get_variable("b3", [n_classes,1], initializer = tf.zeros_initializer())
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3}
    
    return parameters

In [7]:
# Implements forward propagation for the model: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SOFTMAX
# Inputs:
#              X : input dataset placeholder, of shape (input size, number of examples)
#     parameters : python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3"
#                  the shapes are given in initialize_parameters
#
# Returns:
#     Z3 : the output of the last LINEAR unit.
def forward_propagation(X, parameters):

    # Retrieve the parameters from the parameters dictionary
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    
    Z1 = tf.add(tf.matmul(W1, X), b1)                      
    A1 = tf.nn.relu(Z1)                                   
    Z2 = tf.add(tf.matmul(W2, A1), b2)                     
    A2 = tf.nn.relu(Z2)                                   
    Z3 = tf.add(tf.matmul(W3, A2), b3) 
    
    return Z3

In [8]:
# Computes the cost function
# Inputs:
#    Z3 : Output from forward propagation
#    Y : Ground truth labels
# Output:
#    cost : Tensor of the cost function
def compute_cost(Z3, Y):
    
    # to fit the tensorflow requirement for tf.nn.softmax_cross_entropy_with_logits(...,...)
    logits = tf.transpose(Z3)
    labels = tf.transpose(Y)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels = labels))
    
    return cost

In [9]:
# Implements a a three-layer tensorflow neural network: LINEAR->RELU->LINEAR->RELU->LINEAR->SOFTMAX.
def model(train_X, train_labels, cv_X, cv_labels,
          learning_rate = 0.0001, num_epochs = 1500, minibatch_size = 32, print_cost = True):
    
    tf.reset_default_graph()
    costs = [] # To keep track of costs
    
    # Meta data
    (n_x, m) = train_X.shape # n_x = input size, m = number of examples
    n_y = train_labels.shape[0]
    
    # Build graph
    X, Y = create_placeholders(n_x, n_y)
    parameters = initialise_parameters(n_x, n_y)
    Z3 = forward_propagation(X, parameters)
    cost = compute_cost(Z3, Y)
    
    # Define optimiser
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    
    # Initialise all the variables
    init = tf.global_variables_initializer()
    
    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:
        
        # Run the initialisation
        sess.run(init)
        
        # Do the training loop
        for epoch in range(num_epochs):
            
            epoch_cost = 0.                           # Defines a cost related to an epoch
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            minibatches = random_mini_batches(train_X, train_labels, minibatch_size)

            for minibatch in minibatches:

                # Select a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                
                # Run the graph on a minibatch
                _ , minibatch_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})
                
                epoch_cost += minibatch_cost / num_minibatches

            # Print the cost every epoch
            if print_cost == True and epoch % 100 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            if print_cost == True and epoch % 5 == 0:
                costs.append(epoch_cost)
        
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per fives)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

        # lets save the parameters in a variable
        parameters = sess.run(parameters)
        print ("Parameters have been trained!")

        # Calculate the correct predictions
        correct_prediction = tf.equal(tf.argmax(Z3), tf.argmax(Y))

        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

        print ("Train Accuracy:", accuracy.eval({X: train_X, Y: train_labels}))
        print ("CV Accuracy:", accuracy.eval({X: cv_X, Y: cv_labels}))
        
        
        return parameters
    

In [10]:
def main():
    tf.reset_default_graph()
    train_filename = "train.csv"
    test_filename = "test.csv"
    train_X, train_labels, cv_X, cv_labels = load_dataset(train_filename)
    
    print("There are {}, {} train and CV examples respectively.".format(train_X.shape[0], cv_X.shape[0]))
    
    n_classes = 10 
    
    learning_rate = 0.001
    num_epochs = 200
    minibatch_size = 32
    print_cost = True
    
    # E.g. view a digit from the training set and print the corresponding label
    # index = 4
    # plt.imshow(train_X.reshape(train_X.shape[0],28,28)[index])
    # print("Label is {}".format(train_labels[index]))
    
    # Flatten the training, cv and test images
    train_X_flatten = train_X.reshape(train_X.shape[0], -1).T
    cv_X_flatten = cv_X.reshape(cv_X.shape[0], -1).T
    
    # Normalize image vectors
    train_X = train_X_flatten/255.
    cv_X = cv_X_flatten/255.
    
    # Convert labels to one hot matrices
    train_labels = convert_to_one_hot(train_labels, n_classes)
    cv_labels = convert_to_one_hot(cv_labels, n_classes)
    
    # Train model on train data and then test it on the `cv' data
    parameters, accuracy = model(train_X, train_labels, cv_X, cv_labels,
                                   learning_rate = learning_rate, num_epochs = num_epochs, 
                                   minibatch_size = minibatch_size, print_cost = print_cost)
    
    # Load test data, run model and write file needed for Kaggle submission
    
    
    return parameters
    

In [ ]:
parameters = main()

There are 37800, 4200 train and CV examples respectively.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Cost after epoch 0: 0.500099
